### TEST Geography Cleaning Steps

- The goal of this notebook is to test the geography cleaning steps in the code.

### Set Up

In [1]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

pd.set_option('display.max_rows', 500)

### Load CDOT Data

In [6]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format
from cdot_clean_geography_columns import handle_missing_values_in_geography_columns, \
fill_missing_city_values, create_truncated_lat_long, combine_loc1_loc2_alphabetically 

In [7]:
# Load CDOT Data for a list of years

years = [2020, 2024]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list).reset_index()

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2020.xlsx
['SYSTEM', 'RTE', 'SEC', 'MP', 'DATE']
(86304, 85)


/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../convert_old_cdot_format_to_new_format.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = ''
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../convert_old_cdot_format_to_new_format.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = ''
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../convert_old_cdot_format_to_new_format.py:403: Perform

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2024.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(92810, 111)


### Geography Cleaning

In [8]:
print('Loaded Data Size', cdot_pdf_load.shape)

Loaded Data Size (179114, 150)


In [9]:
# STEP 1 - Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_load.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)

display(cdot_pdf_withNans_dedup.head(5))

Data size after drop duplicates (179114, 150)


,index,System Code,Crash Date,Injury 00,Injury 01,Injury 02,Injury 03,Injury 04,Agency Id,Location,...,Wild Animal,Road Contour Curves,Road Contour Grade,Number Killed,Number Injured,Approach Overtaking Turn,Crash Time,Rd_Number,Rd_Section,City_Street
0,0,County Road,2020-01-01,0.0,0.0,0.0,0.0,0.0,CSP,On Roadway,...,NaN,Straight,Level,0.0,0.0,Not Applicable,00:00:00,NaN,SYCAM,NaN
1,1,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,EGLSO,Ran off right side,...,NaN,Curve Unknown Direction,Level,0.0,0.0,Not Applicable,10:00:00,NaN,NIGHT,NaN
2,2,State Highway,2020-01-01,1.0,1.0,0.0,0.0,0.0,CSP,On Roadway,...,ELK,Straight,Level,0.0,1.0,Not Applicable,10:00:00,160.0A,123.1,NaN
3,3,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,ASO,On Roadway,...,NaN,Straight,Level,0.0,0.0,Not Applicable,60:00:00,NaN,JAMIS,NaN
4,4,City Street,2020-01-01,0.0,1.0,0.0,0.0,0.0,DPD,Ran off right side,...,NaN,Straight,Level,0.0,1.0,Not Applicable,13:00:00,NaN,AURAR,NaN


In [10]:
# STEP 2 - fill NaN and null values in geography columns, handle corner cases

# check NaN count by column 
for col in ['City', 'County', 'Crash Year', 'Crash Date', 'Latitude', 'Longitude']:
    print(col, cdot_pdf_withNans_dedup.loc[cdot_pdf_withNans_dedup[col].isna()].shape[0])
print()

cdot_pdf = handle_missing_values_in_geography_columns(cdot_pdf_withNans_dedup)

# check NaN count by column
print('after handle nans')
for col in ['City', 'County', 'Crash Year', 'Crash Date', 'Latitude', 'Longitude']:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])

# confirm no corner cases remain
print(cdot_pdf.loc[cdot_pdf['City'].str.contains('UNKNOWN')].shape)
default_lat = 39.74602
default_long = -104.98877
print(cdot_pdf.loc[(cdot_pdf['Latitude'] == default_lat) & (cdot_pdf['Longitude'] == default_long)].shape)
print(cdot_pdf.loc[(cdot_pdf['Latitude'].round() == 0) & (cdot_pdf['Longitude'].round() == 0)].shape)

City 45346
County 165
Crash Year 97
Crash Date 97
Latitude 5732
Longitude 5730

after handle nans
City 0
City 0
County 0
County 0
Crash Year 97
Crash Year 97
Crash Date 97
Crash Date 97
Latitude 0
Latitude 0
Longitude 0
Longitude 0
(0, 150)
(0, 150)
(0, 150)


In [14]:
# STEP 3 - Fill missing city values 

# filter to El Paso County for time saving **
cdot_pdf_elpaso = cdot_pdf.loc[cdot_pdf['County'] == 'EL PASO']

# remove missing Latitude and Longitude info
cdot_pdf_w_lat_long = cdot_pdf_elpaso.loc[~(cdot_pdf_elpaso['Latitude'] == 999.999000) & ~(cdot_pdf_elpaso['Longitude'] == 999.999000)]

# filter to where city is not provided and see how often we can get the associated city
cdot_pdf_w_latlong_and_no_city = cdot_pdf_w_lat_long.loc[cdot_pdf_w_lat_long['City'] == 'NONE']

# create small test set 
test_set = cdot_pdf_w_latlong_and_no_city.head(10)
display(test_set[['Latitude', 'Longitude', 'City']]) 

# fill missing values
test_set_filled_city = fill_missing_city_values(test_set)

display(test_set_filled_city[['Latitude', 'Longitude', 'City']]) 

,Latitude,Longitude,City
18,38.715241,-104.726856,NONE
178,38.748394,-104.819872,NONE
372,39.120996,-104.865189,NONE
373,39.119588,-104.865425,NONE
374,39.119588,-104.865425,NONE
376,39.116754,-104.865712,NONE
499,39.051470,-104.763841,NONE
606,38.764919,-104.775609,NONE
924,38.889233,-104.964796,NONE
1200,38.764919,-104.775609,NONE


/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../cdot_clean_geography_columns.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdot_pdf[string_geography_cols] = cdot_pdf[string_geography_cols].fillna('NONE')
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/../cdot_clean_geography_columns.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdot_pdf['Latitude'] = cdot_pdf['Latitude'].fillna(999.999000)
/Users/haley.northrup/Desktop/

Geocoding error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=39.119588&lon=-104.865425&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.764919&lon=-104.775609&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


,Latitude,Longitude,City
18,38.715241,-104.726856,FOUNTAIN
178,38.748394,-104.819872,COLORADO SPRINGS
372,39.120996,-104.865189,WOODMOOR
373,39.119588,-104.865425,NONE
374,39.119588,-104.865425,WOODMOOR
376,39.116754,-104.865712,WOODMOOR
499,39.051470,-104.763841,None
606,38.764919,-104.775609,None
924,38.889233,-104.964796,None
1200,38.764919,-104.775609,NONE


In [12]:
# STEP 5 - Create truncated latitude and longitude for analysis 
cdot_pdf = create_truncated_lat_long(cdot_pdf, 5) 

display(cdot_pdf[['Latitude', 'Latitude_5dec', 'Longitude', 'Longitude_5dec']].head(3))

,Latitude,Latitude_5dec,Longitude,Longitude_5dec
0,999.999000,999.99900,999.999000,999.99900
1,999.999000,999.99900,999.999000,999.99900
2,37.204663,37.20466,-107.329465,-107.32946


In [13]:
# STEP 6 -  combine location 1 and location 2
# more useful to understand the intersection vs. location 1 at location 2 being different than location 2 at location 2. 
# *************************
cdot_pdf = combine_loc1_loc2_alphabetically(cdot_pdf) 

# check result
display(cdot_pdf[['Location 1', 'Location 2', 'loc1_first', 'alph_loc1_loc2']].head(5))

,Location 1,Location 2,loc1_first,alph_loc1_loc2
0,SYCAMORE CT,WELLINGTON AVE,True,SYCAMORE CT WELLINGTON AVE
1,NIGHTHAWK CIR,41 NIGHTHAWK CIR,False,41 NIGHTHAWK CIR NIGHTHAWK CIR
2,HWY 160,MM 123,True,HWY 160 MM 123
3,JAMISON DR 15700 E,JASPER CT,True,JAMISON DR 15700 E JASPER CT
4,AURARIA PK,9TH ST,False,9TH ST AURARIA PK
